In [56]:
import cv2
import numpy as np
import dlib
import math
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageEnhance

In [ ]:
# 磨皮(去噪)

def smooth(img,value1=3,value2=1):
    opacity=0.1
    dx=value1*5
    fc=value1*12.5
    temp_img=cv2.bilateralFilter(img,dx,fc,fc)
    temp_img=temp_img-img
    temp_img=cv2.GaussianBlur(temp_img,(2*value2-1,2*value2-1),0)
    temp_img=img+temp_img
    dst=cv2.addWeighted(img,opacity,temp_img,(1-opacity),0)
    return dst

img=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')
smooth=smooth(img)
cv2.imwrite('smooth.jpg',smooth)

In [80]:
# RGB調整

def RGB_adjustment(img,r,g,b):
    (B,G,R)=cv2.split(img)
    B=B+b
    G=G+g
    R=R+r
    cv2.imshow('B', B)
    merged=cv2.merge([B,G,R])
    return merged

img=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')
rgb=RGB_adjustment(img,100,0,0)
cv2.imwrite('rgb.jpg',rgb)

In [15]:
filepath='C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg'
img=Image.open(filepath)

# 亮度
def BrightnessEnhancement(image,brightness):
    enh_bri=ImageEnhance.Brightness(image)
    image_brightened=enh_bri.enhance(brightness)
    return image_brightened

# 對比度
def ContrastEnhancement(image,contrast):
    enh_con=ImageEnhance.Contrast(image)
    image_contrasted=enh_con.enhance(contrast)
    return image_contrasted

# 色度
def ColorEnhancement(image,color): 
    enh_col=ImageEnhance.Color(image)
    image_colored=enh_col.enhance(color)
    return image_colored

# 銳度
def SharpnessEnhancement(image,sharpness):
    enh_sha=ImageEnhance.Sharpness(image)
    image_sharped=enh_sha.enhance(sharpness)
    return image_sharped
    
brightness=BrightnessEnhancement(img,1.2)
contrast=ContrastEnhancement(img,1.2)
color=ColorEnhancement(img,1.5)
sharpness=SharpnessEnhancement(img,3)

brightness.save("brightness.jpg")
contrast.save("contrast.jpg")
color.save("color.jpg")
sharpness.save("sharpness.jpg")

In [ ]:
# 瘦臉

predictor_path='C:/Users/Ying Jhu/Desktop/DIPCV/final project/shape_predictor_68_face_landmarks.dat'
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(predictor_path)
 
def landmark_dec_dlib_fun(img_src):
    img_gray=cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)
    land_marks=[]
    rects=detector(img_gray,0)
    for i in range(len(rects)):
        land_marks_node=np.matrix([[p.x,p.y] for p in predictor(img_gray,rects[i]).parts()])
        # for idx,point in enumerate(land_marks_node):
        #     pos = (point[0,0],point[0,1])
        #     print(idx,pos)
        #     cv2.circle(img_src, pos, 5, color=(0, 255, 0))
        #     font = cv2.FONT_HERSHEY_SIMPLEX
        #     cv2.putText(img_src, str(idx + 1), pos, font, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
        land_marks.append(land_marks_node)
    return land_marks
 
# Interactive Image Warping 局部平移算法
def localTranslationWarp(srcImg,start,end,radius):
    ddradius=float(radius*radius)
    copyImg=np.zeros(srcImg.shape,np.uint8)
    copyImg=srcImg.copy()
    # |m-c|^2
    ddmc=(end[0,0]-start[0,0])*(end[0,0]-start[0,0])+(end[0,1]-start[0,1])*(end[0,1]-start[0,1])
    H,W,C=srcImg.shape
    for i in range(W):
        for j in range(H):
            # 判斷點是否在變形的範圍內
            if math.fabs(i-start[0,0])>radius and math.fabs(j-start[0,1])>radius:
                continue
            distance=(i-start[0,0])*(i-start[0,0])+(j-start[0,1])*(j-start[0,1])
            if (distance<ddradius):
                # 公式
                ratio=(ddradius-distance)/(ddradius-distance+ddmc)
                ratio=ratio*ratio
                UX=i-ratio*(endX-start[0,0])
                UY=j-ratio*(endY-start[0,1])
                value=BilinearInsert(srcImg,UX,UY)
                copyImg[j,i]=value
    return copyImg

# 雙線性插值法
def BilinearInsert(src,ux,uy):
    w,h,c=src.shape
    if c==3:
        x1=int(ux)
        x2=x1+1
        y1=int(uy)
        y2=y1+1
        part1=src[y1,x1].astype(np.float)*(float(x2)-ux)*(float(y2)-uy)
        part2=src[y1,x2].astype(np.float)*(ux-float(x1))*(float(y2)-uy)
        part3=src[y2,x1].astype(np.float)*(float(x2)-ux)*(uy-float(y1))
        part4=src[y2,x2].astype(np.float)*(ux-float(x1))*(uy-float(y1))
        insertValue=part1+part2+part3+part4
        return insertValue.astype(np.int8)

def face_thin(src,left,right):
    landmarks = landmark_dec_dlib_fun(src)
    # 是否偵測到臉
    if len(landmarks)==0:
        return
    for landmarks_node in landmarks:
        left_landmark=landmarks_node[3]
        left_landmark_down=landmarks_node[5]
        right_landmark=landmarks_node[13]
        right_landmark_down=landmarks_node[15]
        endPt=landmarks_node[30]
        # 左臉瘦臉距離
        r_left=np.linalg.norm(left_landmark-left_landmark_down)
        # 右臉瘦臉距離
        r_right=np.linalg.norm(right_landmark-right_landmark_down)
        r_left=left
        r_right=right
        thin_image=localTranslationWarp(src,left_landmark,endPt,r_left)
        thin_image=localTranslationWarp(thin_image,right_landmark,endPt,r_right)
        return thin_image

img=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')
img=face_thin(img,160,60)
cv2.imwrite('thin.jpg',img)

In [ ]:
# 大眼

predictor_path='C:/Users/Ying Jhu/Desktop/DIPCV/final project/shape_predictor_68_face_landmarks.dat'
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(predictor_path)

def landmark_dec_dlib_fun(img_src):
    img_gray=cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)
    land_marks=[]
    rects=detector(img_gray,0)
    for i in range(len(rects)):
        land_marks_node=np.matrix([[p.x,p.y] for p in predictor(img_gray,rects[i]).parts()])
        land_marks.append(land_marks_node)
    return land_marks
 
# 圖像局部縮法算法
def localZoomWarp(srcImg,point,radius,strength):
    ddradius=float(radius*radius)
    copyImg=np.zeros(srcImg.shape,np.uint8)
    copyImg=srcImg.copy()
    H,W,C=srcImg.shape
    for y in range(W):
        for x in range(H):
            distance=(x-point[0,0])*(x-point[0,0])+(y-point[0,1])*(y-point[0,1])
            if (distance<ddradius):
                ratio=1-distance/ddradius
                ratio=1-strength/100*ratio
                UX=(x-point[0,0])*ratio+point[0,0]
                UY=(y-point[0,1])*ratio+point[0,1]
                value=BilinearInsert(srcImg,UX,UY)
                copyImg[y,x]=value
    return copyImg

def BilinearInsert(src,ux,uy):
    w,h,c=src.shape
    if c==3:
        x1=int(ux)
        x2=x1+1
        y1=int(uy)
        y2=y1+1
        part1=src[y1,x1].astype(np.float)*(float(x2)-ux)*(float(y2)-uy)
        part2=src[y1,x2].astype(np.float)*(ux-float(x1))*(float(y2)-uy)
        part3=src[y2,x1].astype(np.float)*(float(x2)-ux)*(uy-float(y1))
        part4 = src[y2,x2].astype(np.float)*(ux-float(x1))*(uy-float(y1))
        insertValue=part1+part2+part3+part4
        return insertValue.astype(np.int8)

def big_eye(src,left_radius,right_radius,strength):
    # radius眼睛放大範圍半徑.strength眼睛放大程度
    landmarks=landmark_dec_dlib_fun(src)
    if len(landmarks)==0:
        return
    for landmarks_node in landmarks:
        left_landmark=landmarks_node[37]
        left_landmark_down=landmarks_node[41]
        left_center=(left_landmark+left_landmark_down)/2
        right_landmark=landmarks_node[43]
        right_landmark_down=landmarks_node[47]
        right_center=(right_landmark+right_landmark_down)/2
        left_radius=left_radius
        right_radius=right_radius
        strength=strength
        eye_image=localZoomWarp(src,left_center,left_radius,strength)
        eye_image=localZoomWarp(eye_image,right_center,right_radius,strength)
    return eye_image

img=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')
img=big_eye(img,50,50,15)
cv2.imwrite('big_eye.jpg',img)

In [14]:
# 美白

def white(img,v1=3,v2=1):
    dx=v1*5
    fc=v1*12.5 
    p=0.1
    temp1=cv2.bilateralFilter(img,dx,fc,fc)
    temp2=cv2.subtract(temp1,img);
    temp2=cv2.add(temp2,(10,10,10,128))
    temp3=cv2.GaussianBlur(temp2,(2*v2-1,2*v2-1),0)
    temp4=cv2.add(img,temp3)
    dst=cv2.addWeighted(img,p,temp4,1-p,0.0)
    dst=cv2.add(dst,(10,10,10,255))
    return dst

white=white(img)
cv2.imwrite('white.jpg',white)

#針對臉失敗
dlib_path='C:/Users/Ying Jhu/Desktop/DIPCV/final project/shape_predictor_81_face_landmarks.dat'
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(dlib_path)

def get_landmark(img):
    faces=detector(img,1)
    shape=predictor(img,faces[0]).parts()
    return np.matrix([[p.x,p.y] for p in shape])

def white(image): 
    image_cp=image.copy()
    color=(200,200,200)
    landmarks=get_landmark(image)
    point1=landmarks[0:16]
    point2=landmarks[75]
    point3=landmarks[71]
    point4=landmarks[76]
    points=np.r_[point1,point2,point3,point4]
    hull=cv2.convexHull(points)
    cv2.drawContours(image,[hull],-1,color,-1)
    cv2.addWeighted(image,0.1,image_cp,0.9,0,image_cp)
    return image_cp

True

In [ ]:
# 紅唇

dlib_path='C:/Users/Ying Jhu/Desktop/DIPCV/final project/shape_predictor_81_face_landmarks.dat'
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(dlib_path)

def get_landmark(img):
    faces=detector(img,1)
    shape=predictor(img,faces[0]).parts()
    return np.matrix([[p.x,p.y] for p in shape])

# r越大越紅，1~255
def rouge(image,r):
    image_cp=image.copy()
    rouge_color=(0,0,r)
    landmarks=get_landmark(image)
    points_1=landmarks[48:68]
    hull=cv2.convexHull(points_1)
    cv2.drawContours(image,[hull],-1,rouge_color,-1)
    cv2.addWeighted(image,0.2,image_cp,0.9,0,image_cp)
    return image_cp

img=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')
mouth_red=rouge(img,255)
#cv2.imwrite('mouth_red.jpg',mouth_red)

In [32]:
# 濾鏡
img_=cv2.imread('C:/Users/Ying Jhu/Desktop/DIPCV/final project/example.jpg')

def old_pic(img):
    rows,cols,channals=img.shape
    for r in range(rows):
        for c in range(cols):
            B=img.item(r,c,0)
            G=img.item(r,c,1)
            R=img.item(r,c,2)
            img[r,c,0]=np.uint8(min(max(0.272*R+0.534*G+0.131*B,0),255))
            img[r,c,1]=np.uint8(min(max(0.349*R+0.686*G+0.168*B,0),255))
            img[r,c,2]=np.uint8(min(max(0.393*R+0.769*G+0.189*B,0),255))
    return img
old=old_pic(img)
cv2.imwrite('old.jpg',old)

def pencil(img):
    dst_gray,dst_color=cv2.pencilSketch(img,sigma_s=100,sigma_r=0.1,shade_factor=0.03)
    return dst_gray
pen=pencil(img_)
cv2.imwrite('pen.jpg',pen)

def cartoon(img):
    dst_comic=cv2.stylization(img,sigma_s=150,sigma_r=0.25)
    return dst_comic
cartoon=cartoon(img)
cv2.imwrite('cartoon.jpg',cartoon)

True